In [ ]:
#Importing relevant Python libraries and modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
#A function to get the minimal distance of a peptide throughout many escape time simulations from the minor groove of a DNA - converting files to dataframes.
#Input: a directory containing the files depicting the minimal distance between the DNA minor groove and the peptide; an index for the set of simulations within the directory
#Output: datafraes containing the minimal distances
def get_mindist_Minor_diff_times(curr_dir, index):

    #SPECIFY THE PATHS TO THE DATA FILES IN THEIR RESPECTIVE DIRECTORY, AS INDICATED BELOW

    #Reading the minimal distance files and defining list to contain the dataframes to be returned
    mindist_dir = curr_dir + '/mindist_Minor_PR_Long_' + np.str(index)
    all_mindist_files = os.listdir(mindist_dir)
    mindist_dfs = []

    #Reading each minimal distance file, converting it to a dataframe to be stored at the dataframe lists
    for file in all_mindist_files:
        lists = []
        y = np.loadtxt(mindist_dir + '/' + file, comments = ['@', '#'], unpack = True)
        for i in range(len(y)):
            lists.append(list(y[i]))

        d = {}
        for i in range(len(lists)):
            d[i] = lists[i]
        df = pd.DataFrame(d)
        df.set_index(0, inplace=True)
        df.columns = ['min_dist']
        mindist_dfs.append(df)

    #Returning the dataframe lists
    return mindist_dfs

#Converting the minimal distance files to dataframes for both A-DNA and B-DNA
B_DNA_mindist_Minor_diff_times = []; A_DNA_mindist_Minor_diff_times = []

#SPECIFY THE PATHS TO THE DATA FILES IN THEIR RESPECTIVE DIRECTORIES, AS INDICATED BELOW

#For A-DNA
for j in range(2):
    A_DNA_mindist_Minor_diff_times.append(get_mindist_Minor_diff_times('/trajectories/yoav/A_DNA_TA_ValX_Close_To_Minor_Groove_Running_Away_Measures_FAR/', j))

#For B-DNA
for j in range(3):
    B_DNA_mindist_Minor_diff_times.append(get_mindist_Minor_diff_times('/trajectories/yoav/B_DNA_TA_ValX_Close_To_Minor_Groove_Running_Away_Measures_FAR/', j))

/tmp/ipykernel_3946446/2785222921.py:8: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mindist_dir = curr_dir + '/mindist_Minor_PR_Long_' + np.str(index)


In [3]:
#A function to get the probability of a peptide to retain the proximity of the minor groove, its mean distance from the groove throughout the simulations and its standard deviation.
#Input: minimal distance dataframe list; the name of the minimal distance column; the distance threshold of defining an escape event; the time threshold of defining an escape event
def get_probability_distribution(dfs, dist_column = 'min_dist', dist_threshold = 0.5, len_thres = 10000):

    #Unifying all dataframes to a single dataframe
    unified_dfs = pd.DataFrame(columns = [dist_column])
    for i in range(len(dfs)):
        for j in range(len(dfs[i])):
            unified_dfs = pd.concat([unified_dfs, dfs[i][j]], axis = 0, ignore_index = False)
    unified_dfs = unified_dfs.copy().loc[unified_dfs.index <= len_thres]
    all_indices = np.unique(unified_dfs.index)

    #Initializing dicts of the values to be returned
    ratios_dict = {}
    mean_dists = {}
    stds_dists = {}

    #For each time frame, calculating the probability of the peptide to retain the proximity of the DNA minor groove (at a distance bigger than the threshold defined earlier), the mean minimal distance, the standard deviation of the minimal distance
    for ind in all_indices:
        curr_unified_dfs = unified_dfs.copy().loc[unified_dfs.copy().index == ind].reset_index()
        curr_mean_dist = np.mean(curr_unified_dfs.copy()[dist_column])
        curr_std_dist = np.std(curr_unified_dfs.copy()[dist_column])
        curr_unified_dfs_lower_than_threshold = curr_unified_dfs.copy().loc[curr_unified_dfs.copy()[dist_column] <= dist_threshold].reset_index()
        curr_ratio = len(curr_unified_dfs_lower_than_threshold.index) / len(curr_unified_dfs.index)
        ratios_dict[ind] = curr_ratio
        mean_dists[ind] = curr_mean_dist
        stds_dists[ind] = curr_std_dist
    
    #Returning the probabilities to retain the proximity of the DNA minor groove, the means and standard deviations of the minimal peptide distance from the DNA minor groove
    return ratios_dict, mean_dists, stds_dists

In [ ]:
#Getting the probabilities to be near the A-/B-DNA minor groove, the means and standard deviations of the minimal peptide distance from the A-/B-DNA minor groove - distance threshold 0.5 nm
A_ratios, A_mean_dists, A_stds_dists = get_probability_distribution(A_DNA_mindist_Minor_diff_times)
B_ratios, B_mean_dists, B_stds_dists = get_probability_distribution(B_DNA_mindist_Minor_diff_times)

#Getting the probabilities to be near the A-/B-DNA minor groove, the means and standard deviations of the minimal peptide distance from the A-/B-DNA minor groove - distance threshold 0.25 nm
A_ratios_025, A_mean_dists_025, A_stds_dists_025 = get_probability_distribution(A_DNA_mindist_Minor_diff_times, dist_threshold = 0.25)
B_ratios_025, B_mean_dists_025, B_stds_dists_025 = get_probability_distribution(B_DNA_mindist_Minor_diff_times, dist_threshold = 0.25)

#Getting the probabilities to be near the A-/B-DNA minor groove, the means and standard deviations of the minimal peptide distance from the A-/B-DNA minor groove - distance threshold 0.375 nm
A_ratios_0375, A_mean_dists_0375, A_stds_dists_0375 = get_probability_distribution(A_DNA_mindist_Minor_diff_times, dist_threshold = 0.375)
B_ratios_0375, B_mean_dists_0375, B_stds_dists_0375 = get_probability_distribution(B_DNA_mindist_Minor_diff_times, dist_threshold = 0.375)

#Getting the probabilities to be near the A-/B-DNA minor groove, the means and standard deviations of the minimal peptide distance from the A-/B-DNA minor groove - distance threshold 0.75 nm
A_ratios_075, A_mean_dists_075, A_stds_dists_075 = get_probability_distribution(A_DNA_mindist_Minor_diff_times, dist_threshold = 0.75)
B_ratios_075, B_mean_dists_075, B_stds_dists_075 = get_probability_distribution(B_DNA_mindist_Minor_diff_times, dist_threshold = 0.75)

#Getting the probabilities to be near the A-/B-DNA minor groove, the means and standard deviations of the minimal peptide distance from the A-/B-DNA minor groove - distance threshold 1 nm
A_ratios_100, A_mean_dists_100, A_stds_dists_100 = get_probability_distribution(A_DNA_mindist_Minor_diff_times, dist_threshold = 1)
B_ratios_100, B_mean_dists_100, B_stds_dists_100 = get_probability_distribution(B_DNA_mindist_Minor_diff_times, dist_threshold = 1)

#Getting the probabilities to be near the A-/B-DNA minor groove, the means and standard deviations of the minimal peptide distance from the A-/B-DNA minor groove - distance threshold 1.5 nm
A_ratios_150, A_mean_dists_150, A_stds_dists_150 = get_probability_distribution(A_DNA_mindist_Minor_diff_times, dist_threshold = 1.5)
B_ratios_150, B_mean_dists_150, B_stds_dists_150 = get_probability_distribution(B_DNA_mindist_Minor_diff_times, dist_threshold = 1.5)

#Getting the probabilities to be near the A-/B-DNA minor groove, the means and standard deviations of the minimal peptide distance from the A-/B-DNA minor groove - distance threshold 2 nm
A_ratios_200, A_mean_dists_200, A_stds_dists_200 = get_probability_distribution(A_DNA_mindist_Minor_diff_times, dist_threshold = 2)
B_ratios_200, B_mean_dists_200, B_stds_dists_200 = get_probability_distribution(B_DNA_mindist_Minor_diff_times, dist_threshold = 2)

/tmp/ipykernel_3946446/1672987821.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unified_dfs = pd.concat([unified_dfs, dfs[i][j]], axis = 0, ignore_index = False)
/tmp/ipykernel_3946446/1672987821.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unified_dfs = pd.concat([unified_dfs, dfs[i][j]], axis = 0, ignore_index = False)
/tmp/ipykernel_3946446/1672987821.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this

In [ ]:
#A function to plot the peptide probability to retain the proximity of the DNA minor groove (at a distance bigger than the threshold) and the minimal peptide distance from the DNA minor groove.
#Input: the probabilities to be near the A-DNA minor groove; the probabilities to be near the B-DNA minor groove; the minimal peptide distances from A-DNA minor groove; the minimal peptide distances from B-DNA minor groove; the distance threshold
#Output: the probabilities of the last time frames, for both A-DNA and B-DNA 
def get_probability_distribution_and_distance_vs_time_plots(A_ratios, B_ratios, A_means, B_means, threshold):

    #Sorting the A-DNA probabilities
    A_lists_r = sorted(A_ratios.items())
    A_x_r, A_y_r = zip(*A_lists_r)

    #Sorting the B-DNA probabilities
    B_lists_r = sorted(B_ratios.items())
    B_x_r, B_y_r = zip(*B_lists_r)

    #Sorting the A-DNA minimal distances
    A_lists_m = sorted(A_means.items())
    A_x_m, A_y_m = zip(*A_lists_m)

    #Sorting the B-DNA minimal distances
    B_lists_m = sorted(B_means.items())
    B_x_m, B_y_m = zip(*B_lists_m)

    #Defining parameters for drawing
    plt.rcParams['pdf.fonttype'] = 42

    #Generating a new figure
    fig, ax = plt.subplots(figsize = (10, 5))

    #Plotting the A-DNA and B-DNA probabilities
    ax.plot(A_x_r, np.log2(A_y_r), c = 'sienna', label = 'A-DNA')
    ax.plot(B_x_r, np.log2(B_y_r), c = 'plum', label = 'B-DNA')

    #Defining parameters for drawing
    ax.set_ylabel('Log(Probability)', fontsize = 25)
    ax.set_xlabel('Time [ns]', fontsize = 25)
    ax.set_xticks(ticks = ax.get_xticks()[1:-1], labels = [int(x / 1000) for x in ax.get_xticks()[1:-1]], fontsize = 20)
    ax.set_yticks(ticks = ax.get_yticks()[1:-1], labels = ax.get_yticklabels()[1:-1], fontsize = 20)
    ax.legend(fontsize = 20)

    #Saving the figure
    fig.savefig('Lower_than_threshold_' + np.str(threshold) + '_Log_probability_distribution.pdf', format = 'pdf')


    #Defining parameters for drawing
    plt.rcParams['pdf.fonttype'] = 42

    #Generating a new figure
    fig, ax = plt.subplots(figsize = (10, 5))

    #Plotting the A-DNA and B-DNA minimal distances and the utilized distance threshold as a dashed black line
    ax.plot(A_x_m, A_y_m, c = 'sienna', label = 'A-DNA')
    ax.plot(B_x_m, B_y_m, c = 'plum', label = 'B-DNA')
    ax.plot(A_x_m, [threshold for i in range(len(A_x_m))], '--k', alpha = 0.65)

    #Defining parameters for drawing
    ax.set_ylabel('Distance [nm]', fontsize = 25)
    ax.set_xlabel('Time [ns]', fontsize = 25)
    ax.set_xticks(ticks = ax.get_xticks()[1:-1], labels = [int(x / 1000) for x in ax.get_xticks()[1:-1]], fontsize = 20)
    ax.set_yticks(ticks = ax.get_yticks()[:-1], labels = ax.get_yticklabels()[:-1], fontsize = 20)
    ax.legend(fontsize = 20)

    #Saving the figure
    fig.savefig('distance_vs_time_' + np.str(threshold) + '.pdf', format = 'pdf')

    #Returning the probabilities related to the last time frames of both A-DNA and B-DNA
    return A_y_r[-1], B_y_r[-1]

In [ ]:
#Defining the used distance thresholds, the A-DNA and B-DNA probabilities and minimal distances
thresholds = [0.25, 0.375, 0.5, 0.75, 1, 1.5, 2]
A_ratios_list = [A_ratios_025, A_ratios_0375, A_ratios, A_ratios_075, A_ratios_100, A_ratios_150, A_ratios_200]
B_ratios_list = [B_ratios_025, B_ratios_0375, B_ratios, B_ratios_075, B_ratios_100, B_ratios_150, B_ratios_200]
A_means_list = [A_mean_dists_025, A_mean_dists_0375, A_mean_dists, A_mean_dists_075, A_mean_dists_100, A_mean_dists_150, A_mean_dists_200]
B_means_list = [B_mean_dists_025, B_mean_dists_0375, B_mean_dists, B_mean_dists_075, B_mean_dists_100, B_mean_dists_150, B_mean_dists_200]

#Finding the probabilities of both A-DNA and B-DNA at the last time frames
A_last_ratios = []; B_last_ratios = []
for i in range(len(thresholds)):
    curr_last_A_ratio, curr_last_B_ratio = get_probability_distribution_and_distance_vs_time_plots(A_ratios_list[i], B_ratios_list[i], A_means_list[i], B_means_list[i], thresholds[i])
    A_last_ratios.append(curr_last_A_ratio)
    B_last_ratios.append(curr_last_B_ratio)

In [ ]:
#Plotting the last probabilities as a function of the threshold

#Importing the matplotlib module
import matplotlib

#Defining parameters for drawing
plt.rcParams['pdf.fonttype'] = 42

#Generating a new figure
fig, ax = plt.subplots(figsize = (6, 6))

#Plotting the last probability of either A-DNA or B-DNA as a function of the distance threshold
sns.scatterplot(ax = ax, x = thresholds, y = A_last_ratios, s = 200, c = 'sienna', label = 'A-DNA')
sns.scatterplot(ax = ax, x = thresholds, y = B_last_ratios, s = 200, c = 'plum', label = 'B-DNA')

#Defining parameters for drawing
ax.set_ylabel('Fraction After 10 ns', fontsize = 25)
ax.set_xlabel('Threshold', fontsize = 25)
ax.set_xticks(ticks = np.log2(thresholds), labels = [x for x in np.round(np.log2(thresholds), 1)], fontsize = 20)
ax.set_yticks(ticks = ax.get_yticks(), labels = ax.get_yticklabels(), fontsize = 20)
ax.legend(fontsize = 20)
ax.set_xscale('log', base = 2)
ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

#Saving the figure
fig.savefig('fractions_10_ns_vs_threshold.pdf', format = 'pdf')